### Импорт библиотек

In [44]:
import random
import pandas as pd
from datetime import datetime, timedelta

### Формирование отчётного периода

In [ ]:
def GetReportingPeriod():
    start_date = datetime(
        year=2006,
        month=4,
        day=1,
        hour=8,
        minute=0,
        second=0
    )

    stop_date = datetime(
        year=2006,
        month=7,
        day=1,
        hour=9,
        minute=0,
        second=0
    )

    date_range = (stop_date - start_date).days
    
    return [start_date + timedelta(days=day) 
            for day in range(date_range) 
            if (start_date + timedelta(days=day)).weekday() not in [5, 6]
    ]

### Генерация сырых данных

Кратко опишем генерацию времени входа и времени выхода. Пусть нормой считается начало дня в 9:00, а конец дня в 18:00, тогда время рабочего дня $t_{норм} = 9$.

Без псевдослучайного смещения имеем нормы входа и выхода 
$$
t_{входа} = 8 \\
t_{выхода} = 8 + t_{норм}
$$

Вводим псевдослучайное смещение, определяемое следующим образом:
$$
0 \leq t_{смещения} \leq 9059 \text{ (в секундах)}
$$

Таким образом, наименьшее смещение - 0 секунд, наибольшее смещение - 2 часа 30 минут и 59 секунд.
Получаем следующие нормы входа и выхода:
$$
t_{входа} = 8 + t_{смещения} \\
t_{выхода} = 8 + t_{норм} + t_{смещения}
$$

Соответственно, имеем следующие возможные значения в формате ЧЧ:ММ:СС:
$$
min(t_{входа}) = 08:00:00, \, max(t_{входа}) = 10:30:59 \\
min(t_{выхода}) = 17:00:00, \, max(t_{выхода}) = 19:30:59
$$

Опишем некоторые интервалы для точности их классификации. Пусть норма начала дня - $09:00:00$, норма окончания дня - $18:00:00$ и $\exists t_{входа}^{i} и t_{выхода}^{i}$, тогда
$$
08:00:00 \leq t_{входа}^{i} \leq 09:00:00 \text{ - норма}, \\
09:00:00 < t_{входа}^{i} \leq 10:30:59 \text{ - опоздание}, \\
17:00:00 \leq t_{выхода}^{i} < 18:00:00 \text{ - ранний выход}, \\
18:00:00 \leq t_{выхода}^{i} \leq 19:30:59 \text{ - норма}. 
$$

Примечание: результирующая структура не содержит аномалий, описанных в пунктах тз после слов "следует учесть".

In [ ]:
full_names = [
    "Семёнов Дмитрий Иванович",
    "Головин Сергей Вальерьевич",
    "Путилов Андрей Маркович",
    "Лапухов Алексей Дмитриевич",
    "Дружинин Георгий Михайлович"
    ]

status = [
    "Вход",
    "Выход",
    "Доступ запрещён"
]

def GetTimeOffset(hours_max_offset=2, minutes_max_offset=30, seconds_max_offset=59) -> timedelta:
    return timedelta(
            hours=random.randint(0, hours_max_offset),
            minutes=random.randint(0, minutes_max_offset),
            seconds=random.randint(0, seconds_max_offset)
        )

def GenerateData(reporting_period : list) -> pd.DataFrame:
    acc = []

    for current_date in reporting_period:
        for name in full_names:
            acc.append({
                "full_name" : name,
                "datetime" : current_date + GetTimeOffset(),
                "status" : status[0]
            })

            acc.append({
                "full_name" : name,
                "datetime" : current_date + timedelta(9) + GetTimeOffset(),
                "status" : status[1]
            })

    return pd.DataFrame(acc)
            
reporting_period = GetReportingPeriod()
data = GenerateData(reporting_period)       

### Аномализация данных

В данном разделе описанны функции, совершающие "иньекции" в полученный на предыдущем этапе датасет. Под "иньекцией" подразумевается вставка аномальных данных или замена сырых данных на аномальные. Для каждого типа аномалии реализована отдельная функция. 

In [ ]:
def GenerateDinner() -> list:
    dinner_names = full_names[:2]
    acc = []

    for current_date in reporting_period:
        dinner_date = current_date.replace(hour=12, minute=45, second=0)

        for name in dinner_names:
            acc.append({
                "full_name" : [name],
                "datetime" : [dinner_date + GetTimeOffset(hours_max_offset=0, minutes_max_offset=30, seconds_max_offset=59)],
                "status" : [status[0]]
            })

            acc.append({
                "full_name" : [name],
                "datetime" : [dinner_date + timedelta(minutes=30) + GetTimeOffset(hours_max_offset=0, minutes_max_offset=30, seconds_max_offset=59)],
                "status" : [status[1]]
            })

    return acc

[{'full_name': ['Семёнов Дмитрий Иванович'], 'datetime': [datetime.datetime(2006, 4, 3, 12, 51, 56)], 'status': ['Вход']}, {'full_name': ['Семёнов Дмитрий Иванович'], 'datetime': [datetime.datetime(2006, 4, 3, 13, 21, 21)], 'status': ['Выход']}, {'full_name': ['Головин Сергей Вальерьевич'], 'datetime': [datetime.datetime(2006, 4, 3, 12, 46, 28)], 'status': ['Вход']}, {'full_name': ['Головин Сергей Вальерьевич'], 'datetime': [datetime.datetime(2006, 4, 3, 13, 19, 59)], 'status': ['Выход']}, {'full_name': ['Семёнов Дмитрий Иванович'], 'datetime': [datetime.datetime(2006, 4, 4, 13, 8, 4)], 'status': ['Вход']}, {'full_name': ['Семёнов Дмитрий Иванович'], 'datetime': [datetime.datetime(2006, 4, 4, 13, 23, 18)], 'status': ['Выход']}, {'full_name': ['Головин Сергей Вальерьевич'], 'datetime': [datetime.datetime(2006, 4, 4, 12, 51, 35)], 'status': ['Вход']}, {'full_name': ['Головин Сергей Вальерьевич'], 'datetime': [datetime.datetime(2006, 4, 4, 13, 33, 28)], 'status': ['Выход']}, {'full_name':

In [66]:

pd.set_option('display.max_rows', None)
print(data)

                       full_name            datetime status
0       Семёнов Дмитрий Иванович 2006-04-03 10:21:33   Вход
1       Семёнов Дмитрий Иванович 2006-04-12 10:21:18  Выход
2     Головин Сергей Вальерьевич 2006-04-03 10:10:29   Вход
3     Головин Сергей Вальерьевич 2006-04-12 08:10:20  Выход
4        Путилов Андрей Маркович 2006-04-03 08:27:00   Вход
5        Путилов Андрей Маркович 2006-04-12 08:16:54  Выход
6     Лапухов Алексей Дмитриевич 2006-04-03 10:06:21   Вход
7     Лапухов Алексей Дмитриевич 2006-04-12 10:05:51  Выход
8    Дружинин Георгий Михайлович 2006-04-03 09:21:59   Вход
9    Дружинин Георгий Михайлович 2006-04-12 08:12:23  Выход
10      Семёнов Дмитрий Иванович 2006-04-04 08:02:51   Вход
11      Семёнов Дмитрий Иванович 2006-04-13 09:14:48  Выход
12    Головин Сергей Вальерьевич 2006-04-04 08:15:55   Вход
13    Головин Сергей Вальерьевич 2006-04-13 10:24:07  Выход
14       Путилов Андрей Маркович 2006-04-04 08:12:17   Вход
15       Путилов Андрей Маркович 2006-04